In [7]:
import os
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from modelClass import Net

### Load training data columns

In [9]:
filePath =  "../../project_course_data/"

colNamesPath = "dataColNames.txt"

with open(filePath + colNamesPath, "r") as f:
    columns = [str(line.strip()) for line in f]

print(columns)


['Label', 'Duration', 'Packets', 'Bytes', 'IGMP', 'TCP', 'UDP', '0', '1900', '22222', '27018', '27025', '27036', '27043', '27047', '27051', '27053', '27057', '27060', '3478', '3480', '34820', '3702', '4070', '44142', '443', '50002', '50012', '50022', '50027', '5228', '5353', '5355', '67', '80', '8009', '1drv', '1.00E+100', '82f3dc', '833aec', 'a104', 'a184', 'a2', 'a23', 'a95', 'aa784e235de7c8b14', 'adobedc', 'akamaitechnologies', 'all', 'amazonaws', 'ams', 'ams1', 'ams15s47', 'ams15s51', 'ams17s02', 'ams17s04', 'ams17s13', 'ams17s17', 'ams58', 'andreas', 'arn', 'arn001', 'arn04', 'arn09s18', 'arn09s19', 'arn09s20', 'arn09s21', 'arn09s22', 'arn09s23', 'arn09s25', 'arn09s26', 'arn09s27', 'arn1', 'arn11s03', 'arn11s04', 'arn11s09', 'arn11s10', 'arn11s11', 'arn11s12', 'arn11s13', 'arn11s14', 'arn2', 'arn54', 'arn56', 'awsglobalaccelerator', 'bc', 'berlin', 'betterttv', 'bkk03s02', 'bl', 'bunnyinfra', 'c061', 'c062', 'c063', 'c064', 'c066', 'c069', 'c073', 'c076', 'c078', 'c079', 'c081', '

In [ ]:
# Load labels dictionary

In [ ]:
# NETWORK LAYER SIZES
input_size = len(columns) - 1
h1 = 4000
h2 = 1000
h3 = 500
h4 = 100
output_size = len(LABELS_DICT)

In [8]:
# Import trained model weights


modelWeights = "modelWeights.pth"

model = Net()

model.load_state_dict(torch.load(filePath + modelWeights))

TypeError: __init__() missing 6 required positional arguments: 'input_size', 'h1', 'h2', 'h3', 'h4', and 'output_size'